<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/master/assignments/assignment_yourname_class7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 5 Assignment: Computer Vision Neural Network**

**Student Name: Your Name**

# Google CoLab Instructions

This assignment will be most straightforward if you use Google CoLab, because it requires both PyTorch and YOLOv5 to be installed. It will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [2]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment Instructions

For this assignment, you will use YOLO running on Google CoLab.  I suggest that you run this assignment on CoLab because the example code below is already setup to get you started with the correct versions of  YOLO on PyTorch.

For this assignment you are provided with 10 image files that contain 10 different webcam pictures taken at the [Venice Sidewalk Cafe](https://www.westland.net/beachcam/) a WebCam that has been in opration since 1996.  You can find the 10 images here:

* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk2.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk3.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk4.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk5.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk6.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk7.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk8.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk9.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk10.jpg

You can see a sample of the WebCam here:

![alt text](https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg)

YOLO does quite well-recognizing objects in this webcam, as the following image illustrates.

![alt text](https://data.heatonresearch.com/data/t81-558/sidewalk/predictions.jpg)

You are to write a script that counts the number of certain objects in each of the images.  Specifically, you are looking for:

* person
* car
* bicycle
* motorbike
* umbrella
* handbag


Your submitted data frame should also contain a column that identifies which image generated each row.  This column should be named **image** and contain integer numbers between 1 and 10.  There should be 10 rows in total.  The complete data frame should look something like this (not necessarily exactly these numbers).

|image|person|car|bicycle|motorbike|umbrella|handbag|
|-|-|-|-|-|-|-|
|1|23|0|3|4|0|0|
|2|27|1|8|2|0|0|
|3|29|0|0|0|3|0|
|...|...|...|...|...|...|...|


The following code sets up YOLO and then dumps the classification information for the first image.  This notebook only serves to get you started.  Read in all ten images and generate a data frame that looks like the following. Use the **submit** function as you did in previous assignments.

### Installing YOLOv5

YOLO is not available directly through either PIP or CONDA.  Additionally, YOLO is not installed in Google CoLab by default. Therefore, whether you wish to use YOLO through CoLab or run it locally, you need to go through several steps to install it.  This section describes the process of installing YOLO.  The same steps apply to either CoLab or a local install.  For CoLab, you must repeat these steps each time the system restarts your virtual environment.  You must perform these steps only once for your virtual Python environment for a local install.  If you are installing locally, make sure to install to the same virtual environment you created for this course.  The following commands install YOLO directly from its GitHub repository.

In [3]:
import sys

!git clone https://github.com/ultralytics/yolov5 --tag 6.2  # clone
!mv /content/6.2 /content/yolov5
%pip install -qr /content/yolov5/requirements.txt  # install
sys.path.insert(0,'/content/yolov5')

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-287-g574331f9 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/201.2 GB disk)


### Running YOLOv5

In addition to the command line execution we just saw, the YOLO library can easily integrate with Python applications.  The following code adds the downloaded YOLOv5 to Python's environment, allowing **yolov5** to be imported like a regular Python library.

In [4]:
import torch

# Model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5n - yolov5x6, custom

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-287-g574331f9 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


I built the following function from the code presented in the course module. The function combines some of the code from the module to accept an image and return what YOLO recognizes. Make sure to use the same two thres_xxx values I provided below to match the results that I got.

In [5]:
from PIL import Image
from io import BytesIO

# Add your solution here, put your results into submit_df

# This is your student key that I emailed to you at the beginnning of the semester.
#key = "5iuwhudihwiao6dsfw7dE2ml08iNfVOg6l0O3M06"  # This is an example key and will not work.
key = "uTtH5yNbPs9tZhhhhdRWsBf9V99RU2iP5cL7F3zH"

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/MyDrive/Colab Notebooks/assignment_yourname_class5.ipynb'  # Google CoLab

# Starter code
i = 1
url = f"https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk{i}.jpg"
response = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
img = Image.open(BytesIO(response.content))

# Inference
results = yolo_model(img)

# Results
df = results.pandas().xyxy[0]

print(df)

submit(source_file=file,data=[df],key=key,no=5)

           xmin        ymin         xmax        ymax  confidence  class  \
0   1232.664185  655.564087  1280.457642  785.814697    0.701470      0   
1    897.502441  542.024292   940.235229  641.454651    0.668886      0   
2    985.573364  506.579315  1029.304932  598.225830    0.643762      0   
3   1052.518799  578.318298  1094.240479  687.382690    0.522934      0   
4   1457.463501  316.237640  1521.751221  354.111542    0.469375      2   
5    964.298828  596.666626  1018.449402  715.114868    0.416467      0   
6    634.272827  443.288177   689.804321  535.599243    0.386331      0   
7    675.782654  624.517822   757.266479  733.891968    0.345984      0   
8    150.537537  280.010315   217.492279  330.766876    0.295688      0   
9   1421.322754  351.651489  1458.115479  384.356110    0.285915      2   
10  1212.849731  361.386353  1283.806519  401.129150    0.259512     25   

        name  
0     person  
1     person  
2     person  
3     person  
4        car  
5     per

# Assignment #5 MyCode

In [7]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
import sys

!git clone https://github.com/ultralytics/yolov5 --tag 6.2  # clone
!mv /content/6.2 /content/yolov5
%pip install -qr /content/yolov5/requirements.txt  # install
sys.path.insert(0,'/content/yolov5')

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-287-g574331f9 Python-3.10.12 torch-2.1.0+cu121 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/107.7 GB disk)


In [ ]:
import torch


yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-287-g574331f9 Python-3.10.12 torch-2.1.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
!mkdir -p /content/images
!wget -O /content/images/sidewalk1.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg
!wget -O /content/images/sidewalk2.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk2.jpg
!wget -O /content/images/sidewalk3.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk3.jpg
!wget -O /content/images/sidewalk4.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk4.jpg
!wget -O /content/images/sidewalk5.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk5.jpg
!wget -O /content/images/sidewalk6.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk6.jpg
!wget -O /content/images/sidewalk7.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk7.jpg
!wget -O /content/images/sidewalk8.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk8.jpg
!wget -O /content/images/sidewalk9.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk9.jpg
!wget -O /content/images/sidewalk10.jpg https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk10.jpg

--2024-02-27 01:22:49--  https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg
Resolving data.heatonresearch.com (data.heatonresearch.com)... 52.84.125.38, 52.84.125.18, 52.84.125.73, ...
Connecting to data.heatonresearch.com (data.heatonresearch.com)|52.84.125.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71055 (69K) [image/jpeg]
Saving to: ‘/content/images/sidewalk1.jpg’

/content/images/sid 100%[===================>]  69.39K  --.-KB/s    in 0.01s   

2024-02-27 01:22:49 (5.25 MB/s) - ‘/content/images/sidewalk1.jpg’ saved [71055/71055]

--2024-02-27 01:22:49--  https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk2.jpg
Resolving data.heatonresearch.com (data.heatonresearch.com)... 52.84.125.38, 52.84.125.18, 52.84.125.73, ...
Connecting to data.heatonresearch.com (data.heatonresearch.com)|52.84.125.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73422 (72K) [image/jpeg]
Saving to: ‘/content/images/side

In [ ]:
!python yolov5/detect.py --weights yolov5s.pt --img 900  \
  --conf 0.1 --iou-thres 0.25 --source /content/images/ --save-txt --save-conf


detect: weights=['yolov5s.pt'], source=/content/images/, data=yolov5/data/coco128.yaml, imgsz=[900, 900], conf_thres=0.1, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-287-g574331f9 Python-3.10.12 torch-2.1.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
WARNING ⚠️ --img-size [900, 900] must be multiple of max stride 32, updating to [928, 928]
image 1/10 /content/images/sidewalk1.jpg: 512x928 12 persons, 7 cars, 1 motorcycle, 1 bus, 1 boat, 2 benchs, 2 umbrellas, 1 suitcase, 821.6ms
image 2/10 /content/images/sidewalk10.jpg: 512x928 29 persons, 1 bicycle, 1 car, 1 dog, 1 suitcase, 701.6ms
image 3/10 /content/im

In [ ]:
import os
import pandas as pd
from collections import Counter

labels_dir = 'yolov5/runs/detect/exp8/labels'

categories = {0: 'person', 2: 'car', 1: 'bicycle', 3: 'motorbike', 26: 'umbrella', 27: 'handbag'}


results = []

for file_name in os.listdir(labels_dir):
    if file_name.endswith('.txt'):

        image_id = int(file_name.split('.')[0].replace('sidewalk', ''))


        with open(os.path.join(labels_dir, file_name), 'r') as file:
            detections = file.readlines()


        counts = Counter()
        for det in detections:
            class_id, conf = det.split()[:2]
            class_id = int(class_id)
            if class_id in categories:
                counts[categories[class_id]] += 1


        result = {'image': image_id}
        result.update(counts)
        results.append(result)

df_columns = ['image', 'person', 'car', 'bicycle', 'motorbike', 'umbrella', 'handbag']
df = pd.DataFrame(results, columns=df_columns).fillna(0).astype(int)

submit_df = df.sort_values('image').reset_index(drop=True)

print(submit_df)


   image  person  car  bicycle  motorbike  umbrella  handbag
0      1      11    5        0          2         0        0
1      2      21    3        0          1         0        0
2      3      20    0        0          0         0        0
3      4      25    0        1          3         0        0
4      5      27    3        4          0         0        0
5      6      36    1        1          2         0        0
6      7      37    0        0          0         0        0
7      8      31    0        1          0         0        0
8      9      36    0        0          0         0        0
9     10      27    1        1          0         0        0


In [3]:

import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

In [ ]:
key = "QGOMi9jY948rtuqknQ9Wb20gQ7BaRlg369Q6fiSX"

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/MyDrive/Colab Notebooks/assignment_solution_class5.ipynb'  # Google CoLab

submit(source_file=file,data=[submit_df],key=key,no=5)

Success: Submitted Assignment 5 for luozihan:
You have submitted this assignment 6 times. (this is fine)
Warnings were identified, these may or may not matter to get full credit for this assignment.


In [1]:
import torch
from PIL import Image
from io import BytesIO
import requests
import pandas as pd
from collections import Counter


yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

results = []


for i in range(1, 11):
    url = f"https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk{i}.jpg"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    img = Image.open(BytesIO(response.content))


    infer_results = yolo_model(img)


    df = infer_results.pandas().xyxy[0]


    categories = {0: 'person', 2: 'car', 1: 'bicycle', 3: 'motorbike', 26: 'umbrella', 27: 'handbag'}
    counts = Counter()
    for index, row in df.iterrows():
        if row['class'] in categories:
            counts[categories[row['class']]] += 1

    result = {'image': i}
    result.update(counts)
    results.append(result)


df_columns = ['image', 'person', 'car', 'bicycle', 'motorbike', 'umbrella', 'handbag']
submit_df = pd.DataFrame(results, columns=df_columns).fillna(0).astype(int)

print(submit_df)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 21.8 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.4s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-2-29 Python-3.10.12 torch-2.1.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 164MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


   image  person  car  bicycle  motorbike  umbrella  handbag
0      1       8    2        0          0         0        0
1      2      12    4        0          0         0        0
2      3       1    0        0          0         0        0
3      4      15    0        0          0         0        0
4      5      13    1        1          0         0        0
5      6       9    2        0          1         0        0
6      7      11    0        0          0         0        0
7      8      12    0        0          0         0        0
8      9       8    0        0          0         0        0
9     10      15    0        0          0         0        0


In [8]:
key = "QGOMi9jY948rtuqknQ9Wb20gQ7BaRlg369Q6fiSX"

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/MyDrive/Colab Notebooks/assignment_solution_class5.ipynb'  # Google CoLab

submit(source_file=file,data=[submit_df],key=key,no=5)

Success: Submitted Assignment 5 for luozihan:
You have submitted this assignment 7 times. (this is fine)
Warnings were identified, these may or may not matter to get full credit for this assignment.
